In [0]:
#download data and pre-trained model 
%sh
rm -rf /dbfs/home/TF/bert
mkdir -p /dbfs/home/TF/bert
cd /dbfs/home/TF/bert/
wget https://github.com/zhixingheyi-tian/Resource/releases/download/TensorFlow/download_glue_data.py
python3 download_glue_data.py --data_dir ./data/ --tasks MRPC
cd /dbfs/home/TF/bert/data/MRPC
wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
unzip uncased_L-12_H-768_A-12.zip
cd /dbfs/home/TF/bert/
wget https://github.com/zhixingheyi-tian/google-research-bert/archive/intel-bert.tar.gz
tar -zxvf intel-bert.tar.gz
cd google-research-bert-intel-bert
ls -l

In [0]:
%sh
# Install necessary package
sudo apt-get update
sudo apt-get install zip -y
sudo apt-get -y install git

# Remove old materials if exist
rm -rf /TF/
mkdir /TF/

# Download google bert
cd /TF/
git clone --single-branch https://github.com/google-research/bert.git && cd bert/
git checkout 88a817c37f788702a363ff935fd173b6dc6ac0d6

# Download IntelAI benchmark
cd /TF/
git clone https://github.com/IntelAI/models.git
cd /TF/models/benchmarks/

export MRPC_DIR=/dbfs/home/TF/bert/data/MRPC
export MODEL_SOURCE=/TF/bert

#Configure the parameter to the same value of Core(s) per socket output by "lscpu" command.
export "cores_per_socket=$(lscpu | awk '/^Core\(s\) per socket/{ print $4 }')"

# Launch Benchmark
python launch_benchmark.py \
  --accuracy-only \
  --verbose \
  --checkpoint $MRPC_DIR/uncased_L-12_H-768_A-12/ \
  --data-location $MRPC_DIR \
  --model-source-dir $MODEL_SOURCE \
  --model-name bert \
  --precision fp32 \
  --mode inference \
  --framework tensorflow \
  --batch-size 32 \
  --num-cores $cores_per_socket \
  --num-inter-threads 1 \
  --num-intra-threads $cores_per_socket \
  --socket-id -1 \
  --output-dir $MRPC_DIR/tmp \
  -- \
  task-name=MRPC \
  max-seq-length=128 \
  learning-rate=2e-5 \
  num_train_epochs=3.0

In [0]:
from __future__ import print_function
from tensorflow.python import _pywrap_util_port
import tensorflow
# Print version, and check whether is intel-optimized
print("tensorflow version: " +tensorflow.__version__)
print(_pywrap_util_port.IsMklEnabled())

In [0]:
%sh
lscpu